In [16]:
# # Install gdown if needed
# # Replace the ID below with your actual file ID from the Drive link
# # (The ID is the long string of random characters in the URL)
# file_id = '1xQvIBwYFTqJRfU77LSqIrRJlsnpdMQGv'
# url = f'https://drive.google.com/uc?id={file_id}'
# output = 'dev_phase.zip'

# !gdown {url} -O {output}

# !unzip {output}

# # Delete __MACOSX directory (if exists) and the dev_phase.zip file (cleanup)
# import os
# import shutil

# if os.path.exists("__MACOSX"):
#     shutil.rmtree("__MACOSX")

# if os.path.exists("dev_phase.zip"):
#     os.remove("dev_phase.zip")

In [17]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from tqdm.auto import tqdm

import torch
import torch.nn as nn
from transformers import AutoModel
from logs import log

In [ ]:
import wandb

# Disable wandb logging for this script
wandb.init(mode="disabled")

# CONFIG
NUM_TYPES = 5
NUM_MANIFESTATIONS = 6
lang = "eng"
trial_id = "MTL_10epochs_full_soft_gating"
model_names = ['bert-base-uncased', "UBC-NLP/MARBERTv2", "microsoft/deberta-v3-base", "FacebookAI/xlm-roberta-large", "0ssamaak0/roberta-base-LEGO_emotions", "FacebookAI/roberta-base", "cardiffnlp/twitter-roberta-base", "cardiffnlp/roberta-base-emotion", "UBC-NLP/ARBERTv2", "cardiffnlp/twitter-roberta-base-offensive", "cardiffnlp/twitter-xlm-roberta-base-sentiment", "cardiffnlp/twitter-roberta-base-hate"]
model_name = model_names[-1]

In [19]:
train_1 = pd.read_csv("./dev_phase/subtask1/train/" + lang + "_augmented.csv")
train_2 = pd.read_csv("./dev_phase/subtask2/train/" + lang + "_augmented.csv")
train_3 = pd.read_csv("./dev_phase/subtask3/train/" + lang + "_augmented.csv")
dev_df = pd.read_csv("./dev_phase/subtask1/dev/" + lang + ".csv")

In [20]:
import pandas as pd

from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
import numpy as np

import torch

from transformers import (
    AutoTokenizer,
    AutoConfig,
    AutoModelForSequenceClassification,
    Trainer,
    TrainingArguments,
    DataCollatorWithPadding
)
from torch.utils.data import Dataset
from tqdm.auto import tqdm

In [ ]:
class PolarizationDataset(torch.utils.data.Dataset):
  def __init__(self,texts,labels,tokenizer,max_length =128):
    self.texts=texts
    self.labels=labels
    self.tokenizer= tokenizer
    self.max_length = max_length # Store max_length

  def __len__(self):
    return len(self.texts)

  def __getitem__(self,idx):
    text=self.texts[idx]
    label=self.labels[idx]
    encoding=self.tokenizer(text,truncation=True,padding=False,max_length=self.max_length,return_tensors='pt')

    # Ensure consistent tensor conversion for all items
    item = {key: encoding[key].squeeze() for key in encoding.keys()}
    item['labels'] = torch.tensor(label, dtype=torch.float)
    return item

In [22]:
from sklearn.model_selection import train_test_split
# Load the tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_name, force_download=True)


train_datasets = []
val_datasets = []

# Prepare label columns separately for each task, fallback to the correct columns per train DataFrame
def get_label_columns(df):
    return [col for col in df.columns if col not in ['id', 'text']]

# Split indices once and reuse for all datasets to ensure same split
n_samples = len(train_1)
indices = np.arange(n_samples)
train_indices, val_indices = train_test_split(
    indices,
    test_size=0.10,
    random_state=42
)
# Merge all datasets on 'id'
merged = train_1.merge(train_2, on=['id', 'text'], how='outer', suffixes=('_1', '_2'))
# For the third, avoid duplicate columns of 'text', so drop redundant one, or merge only on id
merged = merged.merge(train_3, on=['id', 'text'], how='outer', suffixes=('', '_3'))
# Get label columns: all columns excluding 'id' and 'text'
merged_label_columns = get_label_columns(merged)
texts = merged['text'].tolist()
labels = merged[merged_label_columns].values.tolist()
texts_train = [texts[i] for i in train_indices]
texts_val = [texts[i] for i in val_indices]
labels_train = [labels[i] for i in train_indices]
labels_val = [labels[i] for i in val_indices]
train_dataset = PolarizationDataset(texts_train, labels_train, tokenizer)
val_dataset = PolarizationDataset(texts_val, labels_val, tokenizer)

config.json:   0%|          | 0.00/700 [00:00<?, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/150 [00:00<?, ?B/s]

In [23]:
def get_pos_weights(labels_matrix):
    # labels_matrix is a list of lists or numpy array
    labels_np = np.array(labels_matrix)
    num_pos = labels_np.sum(axis=0)
    num_neg = len(labels_np) - num_pos
    
    # Simple ratio: if 10 pos and 90 neg, weight is 9.0
    # Add a small epsilon to avoid division by zero
    weights = num_neg / (num_pos + 1e-5)
    return torch.tensor(weights, dtype=torch.float)

pos_weight_2 = torch.ones([NUM_TYPES]) * 5.0  # Penalize missing a type 5x more
pos_weight_3 = torch.ones([NUM_MANIFESTATIONS]) * 5.0

In [24]:
class SharedMTLModel(nn.Module):
    def __init__(self, model_name, num_types, num_manifestations, pos_weight_2=None, pos_weight_3=None):
        super().__init__()
        self.encoder = AutoModel.from_pretrained(model_name)
        hidden_size = self.encoder.config.hidden_size
        self.num_types = num_types
        self.num_manifestations = num_manifestations

        self.dropout = nn.Dropout(0.2)

        self.head1 = nn.Linear(hidden_size, 1)
        self.head2 = nn.Linear(hidden_size, num_types)
        self.head3 = nn.Linear(hidden_size, num_manifestations)

        self.register_buffer("pos_weight_2", pos_weight_2 if pos_weight_2 is not None else torch.tensor([]))
        self.register_buffer("pos_weight_3", pos_weight_3 if pos_weight_3 is not None else torch.tensor([]))

    def forward(self, input_ids, attention_mask=None, labels=None):
        outputs = self.encoder(input_ids=input_ids, attention_mask=attention_mask)
        H = self.dropout(outputs.last_hidden_state[:, 0, :])

        logits1 = self.head1(H)
        logits2 = self.head2(H)
        logits3 = self.head3(H)
        logits = torch.cat([logits1, logits2, logits3], dim=-1)

        loss = None
        if labels is not None:
            labels = labels.float()
            y1_true = labels[:, :1]
            y2_true = labels[:, 1:1 + self.num_types]
            y3_true = labels[:, 1 + self.num_types:]

            device = logits1.device

            loss1 = nn.BCEWithLogitsLoss()(logits1, y1_true)

            if self.pos_weight_2.numel() > 0:
                loss2 = nn.BCEWithLogitsLoss(pos_weight=self.pos_weight_2.to(device))(logits2, y2_true)
            else:
                loss2 = nn.BCEWithLogitsLoss()(logits2, y2_true)

            if self.pos_weight_3.numel() > 0:
                loss3 = nn.BCEWithLogitsLoss(pos_weight=self.pos_weight_3.to(device))(logits3, y3_true)
            else:
                loss3 = nn.BCEWithLogitsLoss()(logits3, y3_true)

            loss = (loss1 + loss2 + loss3) / 3.0

        return {"loss": loss, "logits": logits}


In [25]:
def compute_pos_weights(df, label_cols):
    labels = df[label_cols].values
    pos = labels.sum(axis=0)
    neg = (labels == 0).sum(axis=0)
    weights = torch.tensor(neg / (pos + 1e-5), dtype=torch.float)
    return weights

pos_weight_2 = compute_pos_weights(train_2, train_2.columns[2:])
pos_weight_3 = compute_pos_weights(train_3, train_3.columns[2:])

model = SharedMTLModel(model_name, NUM_TYPES, NUM_MANIFESTATIONS, pos_weight_2, pos_weight_3)

Some weights of RobertaModel were not initialized from the model checkpoint at cardiffnlp/twitter-roberta-base-hate and are newly initialized: ['pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
def compute_metrics(eval_pred):
    logits = eval_pred.predictions
    labels = eval_pred.label_ids

    if isinstance(logits, tuple):
        logits = logits[0]

    probs = 1 / (1 + np.exp(-logits))
    preds = (probs >= 0.5).astype(int)
    labels = labels.astype(int)

    # Extract Binary Classification (Subtask 1)
    y1_true = labels[:, 0]
    y1_pred = preds[:, 0]

    # --- LOGICAL GATING START ---
    # Create a mask from Subtask 1 predictions
    # Shape becomes (Batch_Size, 1) to broadcast over the other subtasks
    mask = y1_pred[:, None] 
    
    # Extract raw predictions for Subtask 2 & 3
    y2_pred_raw = preds[:, 1:1+NUM_TYPES]
    y3_pred_raw = preds[:, 1+NUM_TYPES:]
    
    # Apply the mask: If y1_pred is 0, force y2 and y3 to be 0
    y2_pred = y2_pred_raw * mask
    y3_pred = y3_pred_raw * mask
    # --- LOGICAL GATING END ---

    # Extract True Labels for Subtask 2 & 3
    y2_true = labels[:, 1:1+NUM_TYPES]
    y3_true = labels[:, 1+NUM_TYPES:]

    return {
        "subtask_1/f1_macro": f1_score(y1_true, y1_pred, average="macro", zero_division=0),
        "subtask_2/f1_macro": f1_score(y2_true, y2_pred, average="macro", zero_division=0),
        "subtask_3/f1_macro": f1_score(y3_true, y3_pred, average="macro", zero_division=0),    }

from transformers import EarlyStoppingCallback

training_args = TrainingArguments(
    output_dir=f"./results/{trial_id}",
    num_train_epochs=10,              # Increase max epochs, let EarlyStopping handle the cut
    learning_rate=2e-5,
    per_device_train_batch_size=32,   # 64 might be too stable? 32 adds some noise (good for regularization)
    per_device_eval_batch_size=16,
    weight_decay=0.01,                # Add Weight Decay!
    eval_strategy="epoch",
    save_strategy="epoch",            # Must save to load best
    load_best_model_at_end=True,      # Automatically load the best checkpoint
    metric_for_best_model="eval_subtask_1/f1_macro", # Optimize for the hardest metric or the main task
    save_total_limit=2,               # Don't fill disk
    logging_steps=50,
)

In [27]:
# Initialize the Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    compute_metrics=compute_metrics,
    data_collator=DataCollatorWithPadding(tokenizer),
    callbacks=[EarlyStoppingCallback(early_stopping_patience=3)] # Stop if no improvement for 3 epochs
)

# Train the model
trainer.train()

# Evaluate the model on the validation set
eval_results = trainer.evaluate()
print(
    "Validation Results:",
    f"\nsubtask_1 f1_macro: {eval_results['eval_subtask_1/f1_macro']:.4f}",
    f"\nsubtask_2 f1_macro: {eval_results['eval_subtask_2/f1_macro']:.4f}",
    f"\nsubtask_3 f1_macro: {eval_results['eval_subtask_3/f1_macro']:.4f}",
)

Epoch,Training Loss,Validation Loss,Subtask 1/f1 Macro,Subtask 2/f1 Macro,Subtask 3/f1 Macro
1,0.748800,0.645815,0.816450,0.319520,0.486326
2,0.641900,0.601729,0.801258,0.408271,0.500152
3,0.507200,0.592193,0.825024,0.445694,0.522047
4,0.448100,0.622306,0.830610,0.430411,0.520266
5,0.366600,0.638689,0.824947,0.466941,0.508951
6,0.306300,0.655902,0.840137,0.488425,0.527493
7,0.290300,0.696550,0.846058,0.506096,0.545292
8,0.264500,0.715739,0.834324,0.524020,0.532121
9,0.236800,0.724865,0.843916,0.512096,0.544302
10,0.233200,0.721451,0.838574,0.517667,0.540448


Validation Results: 
subtask_1 f1_macro: 0.8461 
subtask_2 f1_macro: 0.5061 
subtask_3 f1_macro: 0.5453


# Log Metrics

In [28]:
# Log the experiment results - each subtask separately

# Prepare metadata for the experiment
experiment_metadata = {
    "approach": "MTL_no_gate",
    f"model_{lang}": model_name,
    "learning_rate": training_args.learning_rate,
    "num_train_epochs": training_args.num_train_epochs,
    "per_device_train_batch_size": training_args.per_device_train_batch_size,
    "per_device_eval_batch_size": training_args.per_device_eval_batch_size,
    "num_types": NUM_TYPES,
    "num_manifestations": NUM_MANIFESTATIONS,
    "datasets_merge": datasets_merge,
    "posweight": "True"
}

# Only log eval_f1_macro for each subtask
subtask_1_results = {
    "eval_f1_macro": eval_results.get("eval_subtask_1/f1_macro"),
}
subtask_2_results = {
    "eval_f1_macro": eval_results.get("eval_subtask_2/f1_macro"),
}
subtask_3_results = {
    "eval_f1_macro": eval_results.get("eval_subtask_3/f1_macro"),
}

import json

# Attempt to load existing logs and merge metadata for this trial if present
existing_metadata = {}
try:
    with open("logs.json", "r", encoding="utf-8") as f:
        logs = json.load(f)
        if isinstance(logs, dict):
            logs = [logs]
        for trial in logs:
            if trial.get("trial_id") == trial_id and "metadata" in trial:
                existing_metadata = trial["metadata"].copy()
                break
except (FileNotFoundError, json.JSONDecodeError):
    pass

# Only add/replace model_{lang}, don't overwrite the whole metadata
merged_metadata = dict(existing_metadata)
merged_metadata.update({
    f"model_{lang}": model_name,
    "approach": experiment_metadata["approach"],
    "learning_rate": experiment_metadata["learning_rate"],
    "num_train_epochs": experiment_metadata["num_train_epochs"],
    "per_device_train_batch_size": experiment_metadata["per_device_train_batch_size"],
    "per_device_eval_batch_size": experiment_metadata["per_device_eval_batch_size"],
    "num_types": experiment_metadata["num_types"],
    "num_manifestations": experiment_metadata["num_manifestations"],
    "datasets_merge": experiment_metadata["datasets_merge"]
})

log(
    subtask_name="subtask_1",
    language=lang,
    eval_results=subtask_1_results,
    metadata=merged_metadata,
    trial_id=trial_id
)

# Log subtask_2 and subtask_3 using the same trial_id and do not pass metadata to avoid overwrite
log(
    subtask_name="subtask_2",
    language=lang,
    eval_results=subtask_2_results,
    metadata=None,
    trial_id=trial_id
)

log(
    subtask_name="subtask_3",
    language=lang,
    eval_results=subtask_3_results,
    metadata=None,
    trial_id=trial_id
)

print(f"\n✓ Experiment results logged to logs.json (trial_id: {trial_id})")
print(f"  - subtask_1: {lang}")
print(f"  - subtask_2: {lang}")
print(f"  - subtask_3: {lang}")


✓ Experiment results logged to logs.json (trial_id: MTL_10epochs_full_soft_gating)
  - subtask_1: eng
  - subtask_2: eng
  - subtask_3: eng


# Predict on the dev set

In [29]:
import os

# Load dev 1 and predict all 3 dev sets
dev_1 = pd.read_csv(f"./dev_phase/subtask1/dev/{lang}.csv")
dev_2 = pd.read_csv(f"./dev_phase/subtask2/dev/{lang}.csv")
dev_3 = pd.read_csv(f"./dev_phase/subtask3/dev/{lang}.csv")

# Create dataset from dev 1 texts (all dev sets have same texts)
dev_texts = dev_1['text'].tolist()
# Dummy labels for prediction
dev_dataset = PolarizationDataset(dev_texts, [[0]*12]*len(dev_texts), tokenizer)

# Predict
predictions = trainer.predict(dev_dataset)
logits = predictions.predictions
if isinstance(logits, tuple):
    logits = logits[0]
probs = 1 / (1 + np.exp(-logits))
preds = (probs >= 0.5).astype(int)

# Extract predictions for Subtask 1
polarization_preds = preds[:, 0]

# --- LOGICAL GATING START ---
# Create mask based on Subtask 1 (N, 1)
mask = polarization_preds[:, None]

# Apply mask to Subtasks 2 and 3
# If polarization is 0, these predictions become 0 regardless of model output
types_preds = preds[:, 1:1+NUM_TYPES] * mask
manifestations_preds = preds[:, 1+NUM_TYPES:] * mask

# Create output DataFrames
output_1 = dev_1[['id', 'text']].copy()
output_1['polarization'] = polarization_preds

output_2 = dev_2[['id', 'text']].copy()
type_cols = [col for col in dev_2.columns if col not in ['id', 'text']]
for i, col in enumerate(type_cols):
    output_2[col] = types_preds[:, i]

output_3 = dev_3[['id', 'text']].copy()
manifest_cols = [col for col in dev_3.columns if col not in ['id', 'text']]
for i, col in enumerate(manifest_cols):
    output_3[col] = manifestations_preds[:, i]

# Drop the 'text' column before saving
output_1 = output_1.drop(columns=['text'])
output_2 = output_2.drop(columns=['text'])
output_3 = output_3.drop(columns=['text'])

# Create dir under results with trial_id
os.makedirs(f"./results/{trial_id}", exist_ok=True)
os.makedirs(f"./results/{trial_id}/subtask_1", exist_ok=True)
os.makedirs(f"./results/{trial_id}/subtask_2", exist_ok=True)
os.makedirs(f"./results/{trial_id}/subtask_3", exist_ok=True)

# Save predictions to subtask_ directories
output_1.to_csv(f"./results/{trial_id}/subtask_1/pred_{lang}.csv", index=False)
output_2.to_csv(f"./results/{trial_id}/subtask_2/pred_{lang}.csv", index=False)
output_3.to_csv(f"./results/{trial_id}/subtask_3/pred_{lang}.csv", index=False)

print(f"Predictions saved for all 3 dev sets with Logical Gating applied.")

Predictions saved for all 3 dev sets with Logical Gating applied.


In [30]:
# print
output = output_3
for i in range(len(output)):
    print(",".join([str(output.iloc[i][col]) for col in output.columns]))


eng_f66ca14d60851371f9720aaf4ccd9b58,0,0,0,0,0,0
eng_3a489aa7fed9726aa8d3d4fe74c57efb,0,0,0,0,0,0
eng_95770ff547ea5e48b0be00f385986483,0,0,0,0,0,0
eng_2048ae6f9aa261c48e6d777bcc5b38bf,0,1,1,1,1,1
eng_07781aa88e61e7c0a996abd1e5ea3a20,0,0,0,0,0,0
eng_153d96f9dc27f0602c927223404d94b5,0,0,0,0,0,0
eng_4ab5a4cc5c87d0af9cf4b80c301647bf,0,0,0,0,0,0
eng_e75a95ba52930d6d72d503ab9469eb29,0,0,0,0,0,0
eng_eb8fab668668f9959cafdecbfc0f081a,0,0,0,0,0,0
eng_702724dc168d600e788d775c8e651f36,0,0,0,0,0,0
eng_0efa1a3567443075db38c7ce2dcca571,0,0,0,0,0,0
eng_d08d4243fd2786795df39c1a65dacac7,0,0,0,0,0,0
eng_79fa99ba6989fb61ec127c6c99fc2343,0,0,0,0,0,1
eng_30981038b71c210e97731d90e86038c5,0,0,0,0,0,0
eng_b75e663a6fdc280171b6385b99306a3c,0,0,0,0,0,0
eng_de2baffcfc59b672905e6f2694f672f6,0,0,0,0,0,0
eng_d887794cce49564890a2552cdfb745d2,0,0,0,0,0,0
eng_097b78cf6e209e6778e1fbda10d28b8d,0,0,0,0,0,0
eng_6d29b7c72a091789d06d92157688e07f,0,0,0,0,0,0
eng_00c797e70f1a2f50f1c59655f08581e1,1,1,1,1,1,1
eng_f1e66eab0b9b2c4a